In [1]:
import lilac as ll

ll.set_project_dir('data')

# slimorca = ll.get_dataset('local', 'slimorca')
# slimorca.compute_embedding('gte-small')
# slimorca.select_rows()


/Users/brian/dev/lilac/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = ll.get_dataset('local', 'openorca-10k')


In [3]:
rows = list(ds.select_rows(columns=['question']))


In [8]:
texts = [row['question'] for row in rows[:1000]]


In [5]:
from sentence_transformers import SentenceTransformer


In [6]:
import time
import functools
from lilac.splitters.spacy_splitter import clustering_spacy_chunker
from lilac.embeddings.embedding import compute_split_embeddings
import tqdm


def time_it(model, texts, batch_size, megabatch_size):
  start = time.time()
  embed_fn = functools.partial(model.encode, batch_size=batch_size)
  split_fn = clustering_spacy_chunker
  list(
    tqdm.tqdm(
      compute_split_embeddings(texts, megabatch_size, embed_fn=embed_fn, split_fn=split_fn)))
  end = time.time()
  return end - start


In [9]:
data_rows = []
for model_name in ('thenlper/gte-small', 'thenlper/gte-base', 'all-MiniLM-L6-v2'):
  model = SentenceTransformer(model_name, device='mps')
  for batch_size in (32, 64, 128):
    for megabatch_size in (256, 512, 1024, 2048):
      data_rows.append({
        'model': model_name,
        'batch_size': batch_size,
        'megabatch_size': megabatch_size,
        'time': time_it(model, texts, batch_size, megabatch_size)
      })


1000it [00:09, 101.07it/s]
1000it [00:10, 98.47it/s]
1000it [00:09, 104.17it/s]
1000it [00:09, 104.78it/s]
1000it [00:14, 68.16it/s]
1000it [00:11, 84.71it/s]
1000it [00:11, 89.57it/s]
1000it [00:10, 94.28it/s]
1000it [00:16, 59.55it/s]
1000it [00:14, 71.09it/s]
1000it [00:14, 67.77it/s]
1000it [00:13, 74.63it/s]
1000it [00:28, 35.57it/s]
1000it [00:22, 45.15it/s]
1000it [00:21, 46.38it/s]
1000it [00:21, 46.97it/s]
1000it [00:27, 36.48it/s]
1000it [00:24, 40.10it/s]
1000it [00:24, 40.49it/s]
1000it [00:23, 41.75it/s]
1000it [00:36, 27.54it/s]
1000it [00:31, 31.37it/s]
1000it [00:32, 30.31it/s]
1000it [00:31, 32.17it/s]
1000it [00:06, 163.04it/s]
1000it [00:05, 181.90it/s]
1000it [00:05, 184.14it/s]
1000it [00:05, 185.99it/s]
1000it [00:06, 162.02it/s]
1000it [00:05, 170.55it/s]
1000it [00:05, 172.36it/s]
1000it [00:05, 180.34it/s]
1000it [00:07, 137.87it/s]
1000it [00:06, 151.03it/s]
1000it [00:06, 142.93it/s]
1000it [00:06, 155.97it/s]


In [12]:
import pandas as pd

pd.DataFrame(data_rows).pivot(
  columns='batch_size', index=['model', 'megabatch_size'], values='time').round(2)


batch_size                           32     64     128
model              megabatch_size                     
all-MiniLM-L6-v2   256              6.14   6.17   7.26
                   512              5.50   5.87   6.62
                   1024             5.43   5.80   7.00
                   2048             5.38   5.55   6.41
thenlper/gte-base  256             28.12  27.41  36.32
                   512             22.15  24.94  31.88
                   1024            21.56  24.70  33.00
                   2048            21.29  23.96  31.09
thenlper/gte-small 256              9.90  14.67  16.79
                   512             10.16  11.81  14.07
                   1024             9.60  11.17  14.76
                   2048             9.55  10.61  13.40